In [1]:
import pandas as pd
import draf

COUNTRIES = [
        'AT', 'BE', 'CZ', 'DE', 'DK', 'ES', 'FI', 'FR', 'GB', 'GR', 'HU', 'IE',
        'IT', 'LT', 'NL', 'PL', 'PT', 'RO', 'RS', 'SI'
    ]
FREQS = ["60min", "15min"]
YEARS = range(2017, 2021)

In [2]:
def make_data(year, country, freq):
    df = draf.io.get_emissions(year=year, country=country, freq="60min", additional_info=True, method="_PWL")
    df.index.name = "T"
    if freq == "15min":
        df = draf.hp.upsample(df, year=year)
    df.to_csv(f"data/{freq}/CEFs_{year}_{country}.csv")

def check_length(year, country, freq):
    df = pd.read_csv(f"data/{freq}/CEFs_{year}_{country}.csv")
    steps_per_day = 96 if freq == "15min" else 24
    days_per_year = 366 if year%4==0 else 365
    assert len(df) == days_per_year * steps_per_day

In [ ]:
# EXPENSIVE!

for freq in FREQS:
    for year in YEARS:
        for cy in COUNTRIES:
            make_data(year=year, country=country, freq=freq)

In [3]:
for freq in FREQS:
    for year in YEARS:
        for country in COUNTRIES:
            check_length(year=year, country=country, freq=freq)